In [111]:
import numpy as np
import matplotlib.pyplot as plt

from sklearn import svm

import torch
import torch.nn as nn
from torch.utils.data import DataLoader,sampler
import torchvision

from scipy.linalg import qr,null_space
import math

import warnings
warnings.filterwarnings("ignore", module="sklearn")

In [112]:
train_dataset=torchvision.datasets.MNIST(root='./data/mnist',train=True,
                                         transform=torchvision.transforms.ToTensor(),download=True)
test_dataset=torchvision.datasets.MNIST(root='./data/mnist',train=False,
                                         transform=torchvision.transforms.ToTensor(),download=True)
x_train=train_dataset.data
y_train=train_dataset.targets
x_test=test_dataset.data
y_test=test_dataset.targets

x_train=x_train.view(60000,-1)
x_test=x_test.view(10000,-1)

In [113]:
def Generate_Orthogonal(n):
    H = np.random.randn(n, n)
    Q,_= qr(H)
    return Q

In [114]:
def Solving_Equation(n,Q,b):
    a1=np.matmul(Q,b)
    a=np.zeros([n,n])
    a[0,:]=a1
    return a1,null_space(a).T

In [115]:
def Encoding(n):
    b=np.random.randn(n)
    
    chrome=np.zeros(2*n+1)
    chrome[:n]=b
    chrome[n:2*n-1]=np.random.randint(low=0,high=2,size=(n-1))
    
    num=np.random.randint(1,4)
    if num>=2:
        chrome[2*n-1]=1
    if num!=2:
        chrome[2*n]=1
    
    return chrome

In [116]:
M=50
G=20
Rho=0.9
Mu=0.1
IniN=785
MaxLayer=5
MiniBatch=100

net=nn.Sequential()

#两个序列用于后面随机采样用
Seq_train=np.array(range(len(train_dataset)))
Seq_test=np.array(range(len(test_dataset)))

print(Seq_train)

[    0     1     2 ... 59997 59998 59999]


In [117]:
def Decode(group,chrome):
    n=group.n
    b=chrome[0:n]
    a1,a_remain=Solving_Equation(n,group.Q,b)
    a=np.row_stack((a1,a_remain))
    arch=np.nonzero(chrome[n:2*n-1])
    arch=np.squeeze(arch)
    
    w=np.zeros((len(arch),group.n-1))
    b=np.zeros(len(arch))
    for i in range(len(arch)):
        w[i,:]=a[arch[i],:-1]
        b[i]=a[arch[i],-1]
    act=chrome[2*n-1:2*n+1]
    
    activation=None
    if act[0]==1:
        if act[1]==1:
            activation=torch.relu
        else: 
            activation=torch.sigmoid
    else:
        activation=torch.tanh
    
    return w,b,activation

In [118]:
class Group:
    def __init__(self,n,layer):
        self.layer=layer
        self.data=np.array([Encoding(n) for i in range(M)])
        self.Q=Generate_Orthogonal(n)
        self.n=n
        self.fitness=np.zeros(len(self.data))
        self.dataQ=np.array([]) 
        self.fitnessQ=np.zeros(len(self.data))
        
        self.fitness_elit=0
        self.data_elit=np.zeros(len(self.data))

In [119]:
def Ind_Evaluate(group,chrome,serial_train,serial_test,pre_represent_train,pre_represent_test): 
    w,b,activation=Decode(group,chrome)
    
    past_represent_train=np.matmul(pre_represent_train,w.T)+b
    past_represent_test=np.matmul(pre_represent_test,w.T)+b
    
    past_represent_train=torch.Tensor(past_represent_train)
    past_represent_test=torch.Tensor(past_represent_test)
    
    past_represent_train=activation(past_represent_train).numpy()
    past_represent_test=activation(past_represent_test).numpy()
    
    predictor = svm.SVC(kernel='linear',max_iter=5000)
    predictor.fit(past_represent_train, [y_train[i] for i in serial_train])
    
    result = np.array(predictor.predict(past_represent_test))
    accuracy = np.sum(np.equal(result, [y_test[i] for i in serial_test])) / len(serial_test)
    
    return accuracy

In [120]:
a=torch.Tensor([[1,2],[3,4]]).numpy()
print(a)

[[1. 2.]
 [3. 4.]]


In [121]:
def Group_Evaluate(group,aim):
    
    np.random.shuffle(Seq_train)
    serial_train = Seq_train[0:1000]
    
    np.random.shuffle(Seq_test)
    serial_test = Seq_test[0:1000]
    
    if group.layer>1:
            
        pre_represent_train=net(torch.Tensor([x_train[i].numpy().tolist() for i in serial_train]))
        pre_represent_train=pre_represent_train.detach().numpy()
        
        pre_represent_test=net(torch.Tensor([x_test[i].numpy().tolist() for i in serial_test]))
        pre_represent_test=pre_represent_test.detach().numpy()
    else:
        pre_represent_train=np.array([x_train[i].numpy() for i in serial_train])
        pre_represent_test=np.array([x_test[i].numpy() for i in serial_test])
    
    temp=0
    if aim=='P':
        for ind in group.data:
            group.fitness[temp]=Ind_Evaluate(group,ind,
                                             serial_train,serial_test,pre_represent_train,pre_represent_test)
            temp+=1
    elif aim=='Q':
        for ind in group.dataQ:
            group.fitnessQ[temp]=Ind_Evaluate(group,ind,
                                              serial_train,serial_test,pre_represent_train,pre_represent_test)
            temp+=1

In [122]:
def bin_tour(group):
    index=np.random.choice(M,size=2)
    if group.fitness[index[0]]>group.fitness[index[1]]:
        return group.data[index[0]]
    else:
        return group.data[index[1]]

In [123]:
def bin_tour_Together(group):
    index=np.random.choice(2*M,size=2)
    if group.fitnessQ[index[0]]>group.fitnessQ[index[1]]:
        return index[0]
    else:
        return index[1]

In [124]:
def Sibling(group):
    ind1=bin_tour(group)
    ind2=bin_tour(group)
    if np.random.uniform(0,1)<Rho :
        l1=2*group.n-1
        l2=2
        
        i1=np.random.randint(0,l1)
        i2=np.random.randint(0,l2)
        
        ind1[i1:l1],ind2[i1:l1]=ind2[i1:l1],ind1[i1:l1]
        ind1[l1:l1+i2],ind2[l1:l1+i2]=ind2[l1:l1+i2],ind1[l1:l1+i2]
        
        if ind1[l1]==0 and ind1[l1+1]==0:
            num=np.random.randint(1,4)
            if num>=2:
                ind1[l1]=1
            if num!=2:
                ind1[l1+1]=1
        if ind2[l1]==0 and ind2[l1+1]==0:
            num=np.random.randint(1,4)
            if num>=2:
                ind2[l1]=1
            if num!=2:
                ind2[l1+1]=1       
                
    return ind1,ind2

In [125]:
def SemiPolyMutate(group):
    Itam=20 #分布指数
    for index,ind in enumerate(group.dataQ):
        if np.random.uniform(0,1)<Mu:
            
            for i in range(len(ind)):
                ui=np.random.uniform(0,1)
                if ui<0.5:
                    group.dataQ[index][i]+=math.pow(2*ui,1/(Itam+1))-1
                    
                    #二进制位变异没有找到文章对应描述，故做如下处理。
                    if i>=group.n and group.dataQ[index][i]<0:
                        group.dataQ[index][i]=0
                    else:
                        group.dataQ[index][i]=1

In [126]:
def Operator_Handler(group):
    i=0
    listQ=[]
    while i<M:
        i+=2
        ind1,ind2=Sibling(group)
        listQ.append(ind1)
        listQ.append(ind2)
          
    group.dataQ=np.array(listQ)
    
    SemiPolyMutate(group)

In [127]:
def Select_Together(group):
    group.dataQ=np.row_stack([group.data,group.dataQ])
    group.fitnessQ=np.concatenate([group.fitness,group.fitnessQ])

    index=np.argmax(group.fitnessQ)
    
    group.fitness_elit=group.fitnessQ[index]
    group.data_elit=group.dataQ[index]
    
    i=1
    listP=[]
    listP.append(group.data_elit)
    listPfitness=[]
    listPfitness.append(group.fitness_elit)
    
    while i<M:
        i+=1
        serial=bin_tour_Together(group)
        listP.append(group.dataQ[serial])
        listPfitness.append(group.fitnessQ[serial])
        
    group.data=np.array(listP)
    group.fitness=np.array(listPfitness)

In [129]:
def New_Layer(group):
    w,b,activation=Decode(group,group.data_elit)
    
    if activation==torch.sigmoid: activation=nn.Sigmoid
    elif activation==torch.tanh: activation=nn.Tanh
    elif activation==torch.relu: activation=nn.ReLU
    
    name_nn="Dense"+str(group.layer)
    name_act="Activation"+str(group.layer)
    
    net.add_module(name_nn,nn.Linear(group.n-1,np.shape(w)[0]))
    net.add_module(name_act,activation())  
    print(net)
    #装载参数
    layer_update=(group.layer-1)*2
    net[layer_update].bias.requires_grad=False
    net[layer_update].weight.requires_grad=False
    
    net[layer_update].weight.data=torch.Tensor(w)
    net[layer_update].bias.data=torch.Tensor(b)
    
    net[layer_update].bias.requires_grad=True
    net[layer_update].weight.requires_grad=True
    
    #更新编码长度单位
    return np.shape(w)[0]+1

In [130]:
def Batch_BP(img,label,criterion,optimizer):
    
    out=net(img)

    loss=criterion(out,label)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    return loss.data

In [131]:
def Fine_Tuning(n):
    net.add_module("classifier",nn.Linear(n-1,10))
    net.add_module("softmax",nn.Softmax(dim=1))
    print(net)
    
    criterion=nn.CrossEntropyLoss()
    optimizer=torch.optim.SGD(net.parameters(), lr=0.1)
    
    for epoch in range(1,21):
        train_loss=0
        train_loader=DataLoader(dataset=train_dataset,batch_size=MiniBatch)
        for img,label in train_loader:
            
            img=img.view(MiniBatch, -1)
            train_loss+=Batch_BP(img,label,criterion,optimizer)
            
        train_loss/=len(train_loader)
        print('BP epoch: {}, Train Loss: {:.6f}'.format(epoch, train_loss))

In [132]:
def CCR_Evaluate():
    test_loader=DataLoader(dataset=test_dataset,batch_size=MiniBatch,shuffle=True)
    
    total=0
    correct=0
    
    for images,labels in test_loader:
        images=images.view(MiniBatch,-1)
        labels=labels
        
        out=net(images)
        _,predicts=torch.max(out.data,1)
        total+=labels.size(0)
        correct+=(predicts==labels).sum()
    accuracy=100*correct/total
    return accuracy

In [133]:
if __name__=='__main__':
    n=IniN

    for layer in range(1,MaxLayer):
        group=Group(n,layer)
        Group_Evaluate(group,'P')
        print("Evaluated P")
        
        for generation in range(1,G+1):
            print("generation",generation)
            
            Operator_Handler(group) #Update Q
            
            Group_Evaluate(group,'Q')
            
            Select_Together(group) #Update P and elit, modified Q
            print(group.fitness_elit)
            
        n=New_Layer(group)
    
    Fine_Tuning(n)
    print("Eventual Accuracy is ",CCR_Evaluate())

Evaluated P
generation 1
0.88
Sequential(
  (Dense1): Linear(in_features=784, out_features=376, bias=True)
  (Activation1): ReLU()
)
Evaluated P
generation 1
0.86
Sequential(
  (Dense1): Linear(in_features=784, out_features=376, bias=True)
  (Activation1): ReLU()
  (Dense2): Linear(in_features=376, out_features=203, bias=True)
  (Activation2): ReLU()
)
Evaluated P
generation 1
0.82
Sequential(
  (Dense1): Linear(in_features=784, out_features=376, bias=True)
  (Activation1): ReLU()
  (Dense2): Linear(in_features=376, out_features=203, bias=True)
  (Activation2): ReLU()
  (Dense3): Linear(in_features=203, out_features=110, bias=True)
  (Activation3): ReLU()
)
Evaluated P
generation 1
0.739
Sequential(
  (Dense1): Linear(in_features=784, out_features=376, bias=True)
  (Activation1): ReLU()
  (Dense2): Linear(in_features=376, out_features=203, bias=True)
  (Activation2): ReLU()
  (Dense3): Linear(in_features=203, out_features=110, bias=True)
  (Activation3): ReLU()
  (Dense4): Linear(in_fe